In [ ]:
!pip3 install webdriver_manager
!pip install selenium

In [2]:
# Import packages
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import UnexpectedAlertPresentException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoAlertPresentException
from selenium.common.exceptions import ElementNotInteractableException

import geopandas as gpd

In [3]:
# Input file of parcels with address 'ADDR_FULL'
parcels = gpd.read_file("nwdowntown2.shp")

In [5]:
# Drop parcels with no address
parcels.dropna(subset=['ADDR_FULL'], inplace=True)

In [6]:
# Sort to double check the addresses are valid
sort = parcels.sort_values(by='ADDR_FULL', ascending = False)
sort['ADDR_FULL']

529     922 W CROCKETT ST
528     920 W CROCKETT ST
200       918 W MCGRAW ST
318         918 W HOWE ST
157     918 W HALLADAY ST
              ...        
567    1008 W CROCKETT ST
103     1007 W WHEELER ST
147        1007 W RAYE ST
114    1007 W HALLADAY ST
148        1001 W RAYE ST
Name: ADDR_FULL, Length: 1050, dtype: object

In [7]:
'''
Extract year
Input: address
Returns: year built for the given address
'''
from selenium.webdriver.common.alert import Alert


def extract_year(address):
    # Enter address in the search box
    inputElement = driver.find_element(By.ID, "searchInput")
    inputElement.send_keys(address)

    
    # Click search button
    element = driver.find_element(By.ID, "dijit_form_Button_0_label")
 
    element.click()

    try:
        # Wait for property report link to be clickable
        wait = WebDriverWait(driver, 10)
        element = wait.until(EC.element_to_be_clickable((By.ID, "propertyReportLink")))
    except:
        try:
            # Attempt to switch to an alert
            alert = driver.switch_to.alert
            alert.accept()
            inputElement.clear()
        except NoAlertPresentException:
            # Handle the case where no alert is present
            print("No alert present.")
    
        # Clear inputElement and return None if property report link is not clickable
        #inputElement.clear()
        #return None
    
    # Find property report link
    try:
        element = driver.find_element(By.ID, "propertyReportLink")
    except NoSuchElementException:
        # If property report link is not found, clear inputElement and return None
        inputElement.clear()
        return None

    try:
        element.click()
    except ElementNotInteractableException:
        inputElement.clear()
        return None

    # Switch to the new tab
    driver.switch_to.window(driver.window_handles[-1])

    # List of table IDs to try
    table_ids = [
    "cphContent_DetailsViewPropTypeA",
    "cphContent_DetailsViewPropTypeB",
    "cphContent_DetailsViewPropTypeC",
    "cphContent_DetailsViewPropTypeD",
    "cphContent_DetailsViewPropTypeE",
    "cphContent_DetailsViewPropTypeF",
    "cphContent_DetailsViewPropTypeG",
    "cphContent_DetailsViewPropTypeH",
    "cphContent_DetailsViewPropTypeI",
    "cphContent_DetailsViewPropTypeJ",
    "cphContent_DetailsViewPropTypeK",
    "cphContent_DetailsViewPropTypeL",
    "cphContent_DetailsViewPropTypeM",
    "cphContent_DetailsViewPropTypeN",
    "cphContent_DetailsViewPropTypeO",
    "cphContent_DetailsViewPropTypeP",
    "cphContent_DetailsViewPropTypeQ",
    "cphContent_DetailsViewPropTypeR",
    "cphContent_DetailsViewPropTypeS",
    "cphContent_DetailsViewPropTypeT",
    "cphContent_DetailsViewPropTypeU",
    "cphContent_DetailsViewPropTypeV",
    "cphContent_DetailsViewPropTypeW",
    "cphContent_DetailsViewPropTypeX",
    "cphContent_DetailsViewPropTypeY",
    "cphContent_DetailsViewPropTypeZ"
    ]



    # Try finding the table with different IDs
    year = None
    for table_id in table_ids:
        try:
            table = driver.find_element(By.ID, table_id)
            # Find the 'Year Built' row and extract the year
            rows = table.find_elements(By.TAG_NAME, "tr")
            for row in rows:
                cells = row.find_elements(By.TAG_NAME, "td")
                if len(cells) >= 2 and cells[0].text == 'Year Built':
                    year = cells[1].text
                    break
            if year:
                break  # Break out of loop if year is found
        except NoSuchElementException:
            continue  # Continue to the next table ID if not found

    # Close the second window or tab
    driver.close()

    # Switch back to the original window or tab
    driver.switch_to.window(driver.window_handles[0])

    # Clear inputElement
    inputElement.clear()

    return year

    # If none of the table IDs are found, return None
    if not year:
        return None



In [8]:
# Specify the number of bins/groups
num_bins = 5

# Bin the values into specified number of groups
parcels['Bins'] = pd.cut(parcels.index, bins=num_bins, labels=False)



In [9]:
# Iterate through addresses in the DataFrame and extract years
years = []
bin_nums = [0,1,2,3,4]
for binn in bin_nums:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    # Open a web browser at the following page
    driver.get("https://gismaps.kingcounty.gov/parcelviewer2/")
    for index, row in parcels.iterrows():
        if row['Bins'] == binn :
            address = row['ADDR_FULL']
            year = extract_year(address)
            years.append(year)
    # Close the web driver
    driver.quit()
    print(binn)




# Append the years to the DataFrame as a new column
parcels['year'] = years

#Save output parcel file with year built
parcels.to_file(r"C:\Users\cflemin2\Documents\nwdowntown2YEAR.shp")

0
1
No alert present.
2
3
4
